# Inspecting the eigensolver steps

In [1]:
%load_ext autoreload
%autoreload 2

import sympy as sp
import numpy as np
import matplotlib.pyplot as plt
import json
from pg_utils.pg_model import base, base_utils, core, expansion
from pg_utils import eigen
from IPython.display import display, Math, Markdown

def display_equality(lhs, rhs, _filter=lambda lhs, rhs, idx: True):
    display(*(sp.Eq(lhs_tmp, rhs[idx]) for idx, lhs_tmp in enumerate(lhs) if _filter(lhs_tmp, rhs[idx], idx)))

def display_eq_collection(eqs, _filter=lambda eq, idx: True):
    display(*(eq for idx, eq in enumerate(eqs) if _filter(eq, idx)))

---
## Background field

In [2]:
from pg_utils.pg_model import bg_fields

bg_cfg = bg_fields.BackgroundToroidalQuadrupole()

display_equality(core.U0_vec, bg_cfg.U0_val)
display_equality(core.B0_vec, bg_cfg.B0_val)

Eq(U_s^0(s, \phi, z), 0)

Eq(U_\phi^0(s, \phi, z), 0)

Eq(U_z^0(s, \phi, z), 0)

Eq(B_s^0(s, \phi, z), 0)

Eq(B_\phi^0(s, \phi, z), \gamma*s*(-s**2 - z**2 + 1))

Eq(B_z^0(s, \phi, z), 0)

In [3]:
bg_pg = base_utils.assemble_background(bg_cfg.B0_val)
bg_pg.apply(lambda expr: expr.subs({core.H_s: core.H, core.H_s**2: core.H**2}), inplace=True)
display_equality(core.pgvar_bg, bg_pg)

Eq(\Psi^0(s, \phi), 0)

Eq(\overline{M_{ss}}^0(s, \phi), 0)

Eq(\overline{M_{\phi\phi}}^0(s, \phi), 2*\gamma**2*s**2*H(s)**5/5 + 2*(2*\gamma**2*s**4/3 - 2*\gamma**2*s**2/3)*H(s)**3 + 2*(\gamma**2*s**6 - 2*\gamma**2*s**4 + \gamma**2*s**2)*H(s))

Eq(\overline{M_{s\phi}}^0(s, \phi), 0)

Eq(\widetilde{M_{sz}}^0(s, \phi), 0)

...

Eq(B_\phi^{0+}(s, \phi), 0)

Eq(B_z^{0+}(s, \phi), 0)

Eq(B_s^{0-}(s, \phi), 0)

Eq(B_\phi^{0-}(s, \phi), 0)

Eq(B_z^{0-}(s, \phi), 0)

In [ ]:
bg_placeholders = base.map_collection(core.pgvar_bg, base_utils.extract_symbols(core.pgvar_bg))
placeholder_h = {core.H: sp.Symbol("H")}
for fname in bg_pg._field_names:
    if fname == "Br_b":
        continue
    o_str = sp.latex(sp.Eq(core.pgvar_bg[fname].subs(bg_placeholders), bg_pg[fname].subs(placeholder_h)))
    print(o_str)

---
## Equations

### PG system

In [2]:
# eqs_file = "./out/eigen/Toroidal_Quadrupole/eqs_pg.json"
eqs_file = "./out/eigen/Poloidal_Dipole/eqs_pg.json"
with open(eqs_file, 'r') as fread:
    eqs_pg = base.CollectionPG.deserialize(json.load(fread)["equations"], parser=sp.parse_expr)

Convert to ODE in cylindrical radius $s$ (equiv. convert to frequency-wavenumber domain)

In [8]:
ode_s_fd = eigen.to_fd_ode_pg(eqs_pg, core.pgvar_ptb)

Some further manipulations before output

In [11]:
from pg_utils.sympy_supp import simplify

def common_denom_simplify(expr):
    expr = expr.together()
    num, denom = sp.fraction(expr)
    num = num.subs({core.H: core.H_s}).simplify().factor().subs({core.H_s: core.H, core.H_s**2: core.H**2})
    return num/denom

# eq_grouped = simplify.recursive_collect_tree(eq_rad_std.lhs, (expansion.pgvar_s.Psi, expansion.omega))
# simplify.process_leaf_node(eq_grouped, common_denom_simplify)
# eq_grouped = sp.Eq(simplify.recursive_eval_tree(eq_grouped), eq_rad_std.rhs)

# factor = 1/sp.I/expansion.omega
factor = 1
def factoring(fname, eq):
    if fname != "Psi":
        expr_grouped = (eq.rhs*factor).expand().collect([expansion.pgvar_s.Psi,], evaluate=False)
        for term in expr_grouped:
            expr_grouped[term] = common_denom_simplify(expr_grouped[term])
        return sp.Eq(
            (eq.lhs*factor).expand(), 
            # (eq.rhs*factor).expand().collect(expansion.pgvar_s.Psi)
            factor*sp.Add(*[term*coeff for term, coeff in expr_grouped.items()])
        )
    else:
        expr_grouped = (eq.rhs*factor).expand().collect([var for var in expansion.pgvar_s if var is not None], evaluate=False)
        for term in expr_grouped:
            expr_grouped[term] = common_denom_simplify(expr_grouped[term])
        return sp.Eq((eq.lhs*factor).expand(), sp.Add(*[term*coeff for term, coeff in expr_grouped.items()]))

ode_s_fd.apply(factoring, inplace=True, metadata=True)
display_eq_collection(ode_s_fd)

Eq(-I*\omega*m**2*s*\Psi^{m}(s)/(2*H(s)**3) - I*\omega*m**2*\Psi^{m}(s)/(s*H(s)) + I*\omega*s**2*Derivative(\Psi^{m}(s), s)/H(s)**3 + I*\omega*s*Derivative(\Psi^{m}(s), (s, 2))/H(s) + I*\omega*Derivative(\Psi^{m}(s), s)/H(s), m**2*\widetilde{M_{\phi z}}^{m}(s)/(2*H(s)**2) - m**2*\overline{M_{s\phi}}^{m}(s)/(2*s*H(s)) + 4*I*m*s*(6*s**2 - 5)*B_{z}^{em}(s)/H(s)**2 + I*m*s*B_z^{+m}(s)/H(s)**2 + I*m*s*B_z^{-m}(s)/H(s)**2 - I*m*s*Derivative(\widetilde{M_{sz}}^{m}(s), s)/(2*H(s)**2) + I*m*(-H(s)**2 - 1)*B_s^{+m}(s)/H(s)**3 - I*m*(-H(s)**2 - 1)*B_s^{-m}(s)/H(s)**3 - I*m*Derivative(\overline{M_{\phi\phi}}^{m}(s), s)/(2*H(s)) + I*m*Derivative(\overline{M_{ss}}^{m}(s), s)/(2*H(s)) - I*m*\widetilde{M_{sz}}^{m}(s)/(2*H(s)**2) - I*m*\overline{M_{\phi\phi}}^{m}(s)/(2*s*H(s)) + I*m*\overline{M_{ss}}^{m}(s)/(2*s*H(s)) - 2*s**2*(6*s**2 - 5)*B_{\phi, z}^{em}(s)/H(s)**2 + 2*s*Derivative(B_\phi^{+m}(s), s)/H(s) - 2*s*Derivative(B_\phi^{-m}(s), s)/H(s) - s*Derivative(\overline{M_{s\phi}}^{m}(s), (s, 2))/(2*

Eq(I*\omega*\overline{M_{ss}}^{m}(s), -48*I*m*s*(s - 1)*(s + 1)*Derivative(\Psi^{m}(s), s) + 96*I*m*(2*s**2 - 1)*\Psi^{m}(s))

Eq(I*\omega*\overline{M_{\phi\phi}}^{m}(s), 0)

Eq(I*\omega*\overline{M_{s\phi}}^{m}(s), 24*s**2*(s - 1)*(s + 1)*Derivative(\Psi^{m}(s), (s, 2)) - 24*s*(2*s**2 - 1)*Derivative(\Psi^{m}(s), s))

Eq(I*\omega*\widetilde{M_{sz}}^{m}(s), 6*I*m*(s - 1)*(s + 1)*(12*s**2 - 7)*Derivative(\Psi^{m}(s), s)/H(s) - 6*I*m*(3*s**2 - 2)*(18*s**2 - 7)*\Psi^{m}(s)/(s*H(s)))

...

Eq(I*\omega*B_\phi^{+m}(s), 6*s*Derivative(\Psi^{m}(s), (s, 2)) + 6*(2*s**2 - 1)*Derivative(\Psi^{m}(s), s)/H(s)**2)

Eq(I*\omega*B_z^{+m}(s), 6*I*m*s*Derivative(\Psi^{m}(s), s)/H(s) + 8*I*m*(3*s**2 - 1)*\Psi^{m}(s)/H(s)**3)

Eq(I*\omega*B_s^{-m}(s), 6*I*m*Derivative(\Psi^{m}(s), s) + 12*I*m*(2*s**2 - 1)*\Psi^{m}(s)/(s*H(s)**2))

Eq(I*\omega*B_\phi^{-m}(s), -6*s*Derivative(\Psi^{m}(s), (s, 2)) - 6*(2*s**2 - 1)*Derivative(\Psi^{m}(s), s)/H(s)**2)

Eq(I*\omega*B_z^{-m}(s), 6*I*m*s*Derivative(\Psi^{m}(s), s)/H(s) + 8*I*m*(3*s**2 - 1)*\Psi^{m}(s)/H(s)**3)

In [14]:
placeholders_map = base.map_collection(expansion.pgvar_s, base_utils.extract_symbols(core.pgvar_ptb))
placeholder_h = {core.H: sp.Symbol("H")}
for fname in ode_s_fd._field_names:
    if fname == "Br_b":
        continue
    o_str = sp.latex(ode_s_fd[fname].subs(placeholder_h))
    for var_old, var_new in placeholders_map.items():
        o_str = o_str.replace(sp.latex(var_old), sp.latex(var_new))
    # print(o_str)
    display(Math(o_str))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

Convert directly to reduced form in Fourier domain

In [14]:
reduce_map = {expansion.pgvar_s[fname]: ode_s_fd[fname].rhs if fname not in ("Bp_p", "Bp_m") else sp.S.Zero for fname in ode_s_fd._field_names if fname not in ("Psi", "Br_b")}
ode_psi = sp.Eq((ode_s_fd.Psi.lhs - ode_s_fd.Psi.rhs.subs(reduce_map)).subs({core.H: core.H_s}).doit().subs({core.H_s: core.H, core.H_s**2: core.H**2}).expand(), sp.S.Zero)
(ode_psi.lhs*sp.I*expansion.omega).expand()

I*\gamma**2*m**4*s**5*\Psi^{m}(s)/(6*\omega*H(s)**3) + 8*I*\gamma**2*m**4*s**3*\Psi^{m}(s)/(15*\omega*H(s)) - I*\gamma**2*m**4*s**3*\Psi^{m}(s)/(3*\omega*H(s)**3) - 16*I*\gamma**2*m**4*s*\Psi^{m}(s)/(15*\omega*H(s)) + I*\gamma**2*m**4*s*\Psi^{m}(s)/(6*\omega*H(s)**3) + 8*I*\gamma**2*m**4*\Psi^{m}(s)/(15*\omega*s*H(s)) - 8*I*\gamma**2*m**2*s**5*Derivative(\Psi^{m}(s), (s, 2))/(15*\omega*H(s)) - 5*I*\gamma**2*m**2*s**4*Derivative(\Psi^{m}(s), s)/(3*\omega*H(s)) - 4*I*\gamma**2*m**2*s**3*sqrt(1 - s)*sqrt(s + 1)*\Psi^{m}(s)/(\omega*H(s)**2) + 62*I*\gamma**2*m**2*s**3*\Psi^{m}(s)/(15*\omega*H(s)) + 16*I*\gamma**2*m**2*s**3*Derivative(\Psi^{m}(s), (s, 2))/(15*\omega*H(s)) + 11*I*\gamma**2*m**2*s**2*Derivative(\Psi^{m}(s), s)/(5*\omega*H(s)) + 2*I*\gamma**2*m**2*s*sqrt(1 - s)*sqrt(s + 1)*\Psi^{m}(s)/(\omega*H(s)**2) - 32*I*\gamma**2*m**2*s*\Psi^{m}(s)/(15*\omega*H(s)) - 8*I*\gamma**2*m**2*s*Derivative(\Psi^{m}(s), (s, 2))/(15*\omega*H(s)) - 8*I*\gamma**2*m**2*Derivative(\Psi^{m}(s), s)/(15*\o

In [15]:
def common_denom_simplify(expr):
    expr = expr.together()
    num, denom = sp.fraction(expr)
    num = num.subs({core.H: core.H_s}).simplify().factor().subs({core.H_s: core.H, core.H_s**2: core.H**2}).collect(expansion.m)
    return num/denom.factor()

eq_grouped = simplify.recursive_collect_tree((ode_psi.lhs*expansion.omega**2).expand(), (expansion.pgvar_s.Psi, expansion.omega, expansion.m))
simplify.process_leaf_node(eq_grouped, common_denom_simplify)
eq_grouped = sp.Eq(simplify.recursive_eval_tree(eq_grouped), ode_psi.rhs)
eq_grouped

Eq((-8*\gamma**2*m**2*s*(s - 1)**2*(s + 1)**2/(15*H(s)) + \omega**2*s/H(s))*Derivative(\Psi^{m}(s), (s, 2)) + (-\gamma**2*m**2*(s - 1)*(s + 1)*(25*s**2 - 8)/(15*H(s)) + \omega**2/H(s)**3)*Derivative(\Psi^{m}(s), s) + (-\gamma**2*m**4*(s - 1)**2*(s + 1)**2*(11*s**2 - 16)/(30*s*H(s)**3) - 2*\gamma**2*m**2*s*(1 - s)**(3/2)*(s + 1)**(3/2)/(15*H(s)**2) + \omega**2*m**2*(-H(s)**2 - 1)/(2*s*H(s)**3) - 2*\omega*m*s/(\mathrm{Le}*H(s)**3))*\Psi^{m}(s), 0)

In [80]:
ord_max = 2
leading_cf = ode_psi.lhs.coeff(sp.diff(expansion.pgvar_s.Psi, (core.s, ord_max))).together().subs({core.H_s: core.H, core.H_s**2: core.H**2})
ode_psi = sp.Eq((ode_psi.lhs/leading_cf).expand(), ode_psi.rhs/leading_cf)
sp.Eq(ode_psi.lhs.collect(expansion.pgvar_s.Psi), ode_psi.rhs)

from pg_utils.sympy_supp import simplify

def common_denom_simplify(expr):
    expr = expr.together()
    num, denom = sp.fraction(expr)
    num = num.subs({core.H: core.H_s}).simplify().factor().subs({core.H_s: core.H, core.H_s**2: core.H**2}).collect(expansion.m)
    return num/denom.factor()

eq_grouped = simplify.recursive_collect_tree(ode_psi.lhs, (expansion.pgvar_s.Psi, expansion.omega))
simplify.process_leaf_node(eq_grouped, common_denom_simplify)
eq_grouped = sp.Eq(simplify.recursive_eval_tree(eq_grouped), ode_psi.rhs)
eq_grouped

Eq((-8*\gamma**2*m**2*s**5/(-8*\gamma**2*m**2*s**5 + 16*\gamma**2*m**2*s**3 - 8*\gamma**2*m**2*s + 15*\omega**2*s) + 16*\gamma**2*m**2*s**3/(-8*\gamma**2*m**2*s**5 + 16*\gamma**2*m**2*s**3 - 8*\gamma**2*m**2*s + 15*\omega**2*s) - 8*\gamma**2*m**2*s/(-8*\gamma**2*m**2*s**5 + 16*\gamma**2*m**2*s**3 - 8*\gamma**2*m**2*s + 15*\omega**2*s) + 15*\omega**2*s/(-8*\gamma**2*m**2*s**5 + 16*\gamma**2*m**2*s**3 - 8*\gamma**2*m**2*s + 15*\omega**2*s))*Derivative(\Psi^{m}(s), (s, 2)) + (-25*\gamma**2*m**2*s**4/(-8*\gamma**2*m**2*s**5 + 16*\gamma**2*m**2*s**3 - 8*\gamma**2*m**2*s + 15*\omega**2*s) + 33*\gamma**2*m**2*s**2/(-8*\gamma**2*m**2*s**5 + 16*\gamma**2*m**2*s**3 - 8*\gamma**2*m**2*s + 15*\omega**2*s) - 8*\gamma**2*m**2/(-8*\gamma**2*m**2*s**5 + 16*\gamma**2*m**2*s**3 - 8*\gamma**2*m**2*s + 15*\omega**2*s) + 15*\omega**2*s**2/(-8*\gamma**2*m**2*s**5*H(s)**2 + 16*\gamma**2*m**2*s**3*H(s)**2 - 8*\gamma**2*m**2*s*H(s)**2 + 15*\omega**2*s*H(s)**2) + 15*\omega**2/(-8*\gamma**2*m**2*s**5 + 16*\gamma

In [89]:
o_str = sp.latex(eq_grouped.subs(placeholder_h))
for var_old, var_new in placeholders_map.items():
    o_str = o_str.replace(sp.latex(var_old), sp.latex(var_new))
print(o_str)

\left(- \frac{8 \gamma^{2} m^{2} s \left(s - 1\right)^{2} \left(s + 1\right)^{2}}{15 H} + \frac{\omega^{2} s}{H}\right) \frac{d^{2}}{d s^{2}} \psi + \left(- \frac{\gamma^{2} m^{2} \left(s - 1\right) \left(s + 1\right) \left(25 s^{2} - 8\right)}{15 H} + \frac{\omega^{2}}{H^{3}}\right) \frac{d}{d s} \psi + \left(- \frac{2 \gamma^{2} m^{2} s \left(1 - s\right)^{\frac{3}{2}} \left(s + 1\right)^{\frac{3}{2}}}{15 H^{2}} - \frac{\gamma^{2} m^{4} \left(s - 1\right)^{2} \left(s + 1\right)^{2} \cdot \left(11 s^{2} - 16\right)}{30 H^{3} s} + \frac{\omega^{2} m^{2} \left(- H^{2} - 1\right)}{2 H^{3} s} - \frac{2 \omega m s}{H^{3} \mathrm{Le}}\right) \psi = 0


### Transformed variables

In [ ]:
eqs_file = "./out/eigen/Poloidal_Dipole/eqs_cg.json"
with open(eqs_file, 'r') as fread:
    eqs_cg = base.CollectionConjugate.deserialize(json.load(fread)["equations"], parser=sp.parse_expr)
display_eq_collection(eqs_cg)
# eqs_cg.Psi

In [3]:
ode_s_fd_cg = eigen.to_fd_ode_cg(eqs_cg, core.cgvar_ptb)

Some further manipulations before output

In [4]:
from pg_utils.sympy_supp import simplify

def common_denom_simplify(expr):
    expr = expr.together()
    num, denom = sp.fraction(expr)
    num = num.subs({core.H: core.H_s}).simplify().factor().subs({core.H_s: core.H, core.H_s**2: core.H**2})
    return num/denom

# eq_grouped = simplify.recursive_collect_tree(eq_rad_std.lhs, (expansion.pgvar_s.Psi, expansion.omega))
# simplify.process_leaf_node(eq_grouped, common_denom_simplify)
# eq_grouped = sp.Eq(simplify.recursive_eval_tree(eq_grouped), eq_rad_std.rhs)

# factor = 1/sp.I/expansion.omega
factor = 1
def factoring(fname, eq):
    if fname != "Psi":
        expr_grouped = (eq.rhs*factor).expand().collect([expansion.cgvar_s.Psi,], evaluate=False)
        for term in expr_grouped:
            expr_grouped[term] = common_denom_simplify(expr_grouped[term])
        return sp.Eq(
            (eq.lhs*factor).expand(), 
            # (eq.rhs*factor).expand().collect(expansion.pgvar_s.Psi)
            factor*sp.Add(*[term*coeff for term, coeff in expr_grouped.items()])
        )
    else:
        expr_grouped = (eq.rhs*factor).expand().collect([var for var in expansion.cgvar_s if var is not None], evaluate=False)
        for term in expr_grouped:
            expr_grouped[term] = common_denom_simplify(expr_grouped[term])
        return sp.Eq((eq.lhs*factor).expand(), sp.Add(*[term*coeff for term, coeff in expr_grouped.items()]))

ode_s_fd_cg.apply(factoring, inplace=True, metadata=True)
display_eq_collection(ode_s_fd_cg)

Eq(-I*\omega*m**2*s*\Psi^{m}(s)/(2*H(s)**3) - I*\omega*m**2*\Psi^{m}(s)/(s*H(s)) + I*\omega*s**2*Derivative(\Psi^{m}(s), s)/H(s)**3 + I*\omega*s*Derivative(\Psi^{m}(s), (s, 2))/H(s) + I*\omega*Derivative(\Psi^{m}(s), s)/H(s), 4*I*m*s*(6*s**2 - 5)*B_{z}^{em}(s)/H(s)**2 + I*m*s*B_z^{+m}(s)/H(s)**2 + I*m*s*B_z^{-m}(s)/H(s)**2 - sqrt(2)*I*m*s*Derivative(\widetilde{M_{z+}}^m(s), s)/(4*H(s)**2) - sqrt(2)*I*m*s*Derivative(\widetilde{M_{z-}}^m(s), s)/(4*H(s)**2) + sqrt(2)*I*m*(m - 1)*\widetilde{M_{z-}}^m(s)/(4*H(s)**2) - sqrt(2)*I*m*(m + 1)*\widetilde{M_{z+}}^m(s)/(4*H(s)**2) - I*m*(m - 2)*\overline{M_-}^m(s)/(4*s*H(s)) + I*m*(m + 2)*\overline{M_+}^m(s)/(4*s*H(s)) + sqrt(2)*I*s**2*(6*s**2 - 5)*B_{+, z}^{em}(s)/H(s)**2 - sqrt(2)*I*s**2*(6*s**2 - 5)*B_{-, z}^{em}(s)/H(s)**2 - sqrt(2)*I*s*Derivative(B_+^{+m}(s), s)/H(s) + sqrt(2)*I*s*Derivative(B_+^{-m}(s), s)/H(s) + sqrt(2)*I*s*Derivative(B_-^{+m}(s), s)/H(s) - sqrt(2)*I*s*Derivative(B_-^{-m}(s), s)/H(s) + I*s*Derivative(\overline{M_+}^m(s), (s,

Eq(I*\omega*\overline{M_1}^m(s), -24*I*m*s*(s - 1)*(s + 1)*Derivative(\Psi^{m}(s), s) + 48*I*m*(2*s**2 - 1)*\Psi^{m}(s))

Eq(I*\omega*\overline{M_+}^m(s), 48*I*m*(2*s**2 - 1)*\Psi^{m}(s) + 24*I*s**2*(s - 1)*(s + 1)*Derivative(\Psi^{m}(s), (s, 2)) - 24*I*s*(m*s**2 - m + 2*s**2 - 1)*Derivative(\Psi^{m}(s), s))

Eq(I*\omega*\overline{M_-}^m(s), 48*I*m*(2*s**2 - 1)*\Psi^{m}(s) - 24*I*s**2*(s - 1)*(s + 1)*Derivative(\Psi^{m}(s), (s, 2)) - 24*I*s*(m*s**2 - m - 2*s**2 + 1)*Derivative(\Psi^{m}(s), s))

Eq(I*\omega*\widetilde{M_{z+}}^m(s), -3*sqrt(2)*I*m*(3*s**2 - 2)*(18*s**2 - 7)*\Psi^{m}(s)/(s*H(s)) - 3*sqrt(2)*I*s*(s - 1)*(s + 1)*(9*s**2 - 7)*Derivative(\Psi^{m}(s), (s, 2))/H(s) + 3*sqrt(2)*I*(12*m*s**4 - 19*m*s**2 + 7*m + 18*s**4 - 23*s**2 + 7)*Derivative(\Psi^{m}(s), s)/H(s))

...

Eq(I*\omega*B_-^{+m}(s), -6*sqrt(2)*I*m*(2*s**2 - 1)*\Psi^{m}(s)/(s*H(s)**2) - 3*sqrt(2)*I*s*Derivative(\Psi^{m}(s), (s, 2)) + 3*sqrt(2)*I*(m*s**2 - m - 2*s**2 + 1)*Derivative(\Psi^{m}(s), s)/H(s)**2)

Eq(I*\omega*B_z^{+m}(s), 6*I*m*s*Derivative(\Psi^{m}(s), s)/H(s) + 8*I*m*(3*s**2 - 1)*\Psi^{m}(s)/H(s)**3)

Eq(I*\omega*B_+^{-m}(s), 6*sqrt(2)*I*m*(2*s**2 - 1)*\Psi^{m}(s)/(s*H(s)**2) - 3*sqrt(2)*I*s*Derivative(\Psi^{m}(s), (s, 2)) - 3*sqrt(2)*I*(m*s**2 - m + 2*s**2 - 1)*Derivative(\Psi^{m}(s), s)/H(s)**2)

Eq(I*\omega*B_-^{-m}(s), 6*sqrt(2)*I*m*(2*s**2 - 1)*\Psi^{m}(s)/(s*H(s)**2) + 3*sqrt(2)*I*s*Derivative(\Psi^{m}(s), (s, 2)) - 3*sqrt(2)*I*(m*s**2 - m - 2*s**2 + 1)*Derivative(\Psi^{m}(s), s)/H(s)**2)

Eq(I*\omega*B_z^{-m}(s), 6*I*m*s*Derivative(\Psi^{m}(s), s)/H(s) + 8*I*m*(3*s**2 - 1)*\Psi^{m}(s)/H(s)**3)

In [26]:
forcing = ode_s_fd_cg.M_zp.rhs.subs(
    {expansion.cgvar_s.Psi: core.s**expansion.m*core.H**3*sp.jacobi(expansion.n, sp.Rational(3, 2), expansion.m, 2*core.s**2 - 1)})

In [28]:
forcing = forcing.doit().subs({sp.diff(core.H, core.s): -core.s/core.H}).doit().subs({sp.diff(core.H, core.s): -core.s/core.H})

In [32]:
forcing.expand()

-108*sqrt(2)*I*m**2*s**7*s**m*H(s)**2*jacobi(n - 2, 7/2, m + 2, 2*s**2 - 1) + 192*sqrt(2)*I*m**2*s**5*s**m*H(s)**2*jacobi(n - 2, 7/2, m + 2, 2*s**2 - 1) - 36*sqrt(2)*I*m**2*s**5*s**m*H(s)**2*jacobi(n - 1, 5/2, m + 1, 2*s**2 - 1) + 9*sqrt(2)*I*m**2*s**3*s**m*H(s)**2*jacobi(n, 3/2, m, 2*s**2 - 1) - 84*sqrt(2)*I*m**2*s**3*s**m*H(s)**2*jacobi(n - 2, 7/2, m + 2, 2*s**2 - 1) + 78*sqrt(2)*I*m**2*s**3*s**m*H(s)**2*jacobi(n - 1, 5/2, m + 1, 2*s**2 - 1) - 9*sqrt(2)*I*m**2*s*s**m*H(s)**2*jacobi(n, 3/2, m, 2*s**2 - 1) - 42*sqrt(2)*I*m**2*s*s**m*H(s)**2*jacobi(n - 1, 5/2, m + 1, 2*s**2 - 1) - 216*sqrt(2)*I*m*n*s**7*s**m*H(s)**2*jacobi(n - 2, 7/2, m + 2, 2*s**2 - 1) + 384*sqrt(2)*I*m*n*s**5*s**m*H(s)**2*jacobi(n - 2, 7/2, m + 2, 2*s**2 - 1) - 36*sqrt(2)*I*m*n*s**5*s**m*H(s)**2*jacobi(n - 1, 5/2, m + 1, 2*s**2 - 1) - 168*sqrt(2)*I*m*n*s**3*s**m*H(s)**2*jacobi(n - 2, 7/2, m + 2, 2*s**2 - 1) + 78*sqrt(2)*I*m*n*s**3*s**m*H(s)**2*jacobi(n - 1, 5/2, m + 1, 2*s**2 - 1) - 42*sqrt(2)*I*m*n*s*s**m*H(s)**2*jac

In [37]:
forcing.expand().collect(core.H)

324*sqrt(2)*I*m*s**7*s**m*jacobi(n - 1, 5/2, m + 1, 2*s**2 - 1) + 54*sqrt(2)*I*m*s**5*s**m*jacobi(n, 3/2, m, 2*s**2 - 1) - 576*sqrt(2)*I*m*s**5*s**m*jacobi(n - 1, 5/2, m + 1, 2*s**2 - 1) - 117*sqrt(2)*I*m*s**3*s**m*jacobi(n, 3/2, m, 2*s**2 - 1) + 252*sqrt(2)*I*m*s**3*s**m*jacobi(n - 1, 5/2, m + 1, 2*s**2 - 1) + 63*sqrt(2)*I*m*s*s**m*jacobi(n, 3/2, m, 2*s**2 - 1) + 324*sqrt(2)*I*n*s**7*s**m*jacobi(n - 1, 5/2, m + 1, 2*s**2 - 1) - 576*sqrt(2)*I*n*s**5*s**m*jacobi(n - 1, 5/2, m + 1, 2*s**2 - 1) + 252*sqrt(2)*I*n*s**3*s**m*jacobi(n - 1, 5/2, m + 1, 2*s**2 - 1) + 810*sqrt(2)*I*s**7*s**m*jacobi(n - 1, 5/2, m + 1, 2*s**2 - 1) - 81*sqrt(2)*I*s**5*s**m*jacobi(n, 3/2, m, 2*s**2 - 1) - 1440*sqrt(2)*I*s**5*s**m*jacobi(n - 1, 5/2, m + 1, 2*s**2 - 1) + 63*sqrt(2)*I*s**3*s**m*jacobi(n, 3/2, m, 2*s**2 - 1) + 630*sqrt(2)*I*s**3*s**m*jacobi(n - 1, 5/2, m + 1, 2*s**2 - 1) + (-81*sqrt(2)*I*s**7*s**m*jacobi(n, 3/2, m, 2*s**2 - 1) + 144*sqrt(2)*I*s**5*s**m*jacobi(n, 3/2, m, 2*s**2 - 1) - 63*sqrt(2)*I*s**3*s

In [22]:
placeholders_map = base.map_collection(expansion.cgvar_s, base_utils.extract_symbols(core.cgvar_ptb))
placeholder_h = {core.H: sp.Symbol("H")}
for fname in ode_s_fd_cg._field_names:
    if fname == "Br_b":
        continue
    o_str = sp.latex(ode_s_fd_cg[fname].subs(placeholder_h))
    for var_old, var_new in placeholders_map.items():
        o_str = o_str.replace(sp.latex(var_old), sp.latex(var_new))
    # print(o_str)
    display(Math(o_str))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [23]:
map_cg_pg = core.map_conjugate_to_pg(core.cgvar_ptb, core.pgvar_ptb)

In [26]:
eqs_pg_from_cg = eqs_cg.apply(lambda eq: eq.subs(map_cg_pg))
eqs_pg_from_cg = core.conjugate_to_PG(eqs_pg_from_cg).apply(lambda eq: eq.doit().expand())

In [27]:
display_eq_collection(eqs_pg_from_cg)

Eq(s**2*Derivative(\psi(s, \phi, t), s, t)/H(s)**3 + s*Derivative(\psi(s, \phi, t), (s, 2), t)/H(s) + s*Derivative(\psi(s, \phi, t), (\phi, 2), t)/(2*H(s)**3) + Derivative(\psi(s, \phi, t), s, t)/H(s) + Derivative(\psi(s, \phi, t), (\phi, 2), t)/(s*H(s)), -12*s**4*b_{\phi, z}^e(s, \phi, t)/H(s)**2 - 3*s**4*Derivative(b_s^+(s, \phi, t), \phi)/H(s)**3 + 3*s**4*Derivative(b_s^-(s, \phi, t), \phi)/H(s)**3 - 3*s**3*Derivative(b_z^+(s, \phi, t), \phi)/H(s)**2 - 3*s**3*Derivative(b_z^-(s, \phi, t), \phi)/H(s)**2 + 24*s**3*Derivative(b_{z}^e(s, \phi, t), \phi)/H(s)**2 - 3*s**2*Derivative(b_s^+(s, \phi, t), \phi)/H(s) + 3*s**2*Derivative(b_s^-(s, \phi, t), \phi)/H(s) + 10*s**2*b_{\phi, z}^e(s, \phi, t)/H(s)**2 + 2*s**2*Derivative(b_s^+(s, \phi, t), \phi)/H(s)**3 - 2*s**2*Derivative(b_s^-(s, \phi, t), \phi)/H(s)**3 - 3*s*Derivative(b_z^+(s, \phi, t), \phi) - 3*s*Derivative(b_z^-(s, \phi, t), \phi) - s*Derivative(\overline{m_{s\phi}}(s, \phi, t), (s, 2))/(2*H(s)) + 2*s*Derivative(b_\phi^+(s, \phi

Eq(Derivative(\overline{m_{ss}}(s, \phi, t), t), -48*s**3*Derivative(\psi(s, \phi, t), \phi, s) + 192*s**2*Derivative(\psi(s, \phi, t), \phi) + 48*s*Derivative(\psi(s, \phi, t), \phi, s) - 96*Derivative(\psi(s, \phi, t), \phi))

Eq(Derivative(\overline{m_{\phi\phi}}(s, \phi, t), t), 0)

Eq(Derivative(\overline{m_{s\phi}}(s, \phi, t), t), 24*s**4*Derivative(\psi(s, \phi, t), (s, 2)) - 48*s**3*Derivative(\psi(s, \phi, t), s) - 24*s**2*Derivative(\psi(s, \phi, t), (s, 2)) + 24*s*Derivative(\psi(s, \phi, t), s))

Eq(Derivative(\widetilde{m_{sz}}(s, \phi, t), t), 72*s**4*Derivative(\psi(s, \phi, t), \phi, s)/H(s) - 324*s**3*Derivative(\psi(s, \phi, t), \phi)/H(s) - 114*s**2*Derivative(\psi(s, \phi, t), \phi, s)/H(s) + 342*s*Derivative(\psi(s, \phi, t), \phi)/H(s) + 42*Derivative(\psi(s, \phi, t), \phi, s)/H(s) - 84*Derivative(\psi(s, \phi, t), \phi)/(s*H(s)))

...

Eq(Derivative(b_\phi^+(s, \phi, t), t), 6*s**2*Derivative(\psi(s, \phi, t), s)/H(s)**2 + 6*s*Derivative(\psi(s, \phi, t), (s, 2)) - 6*Derivative(\psi(s, \phi, t), s))

Eq(Derivative(b_z^+(s, \phi, t), t), 6*s**2*Derivative(\psi(s, \phi, t), \phi)/H(s)**3 + 6*s*Derivative(\psi(s, \phi, t), \phi, s)/H(s) - 18*Derivative(\psi(s, \phi, t), \phi)/H(s) + 10*Derivative(\psi(s, \phi, t), \phi)/H(s)**3)

Eq(Derivative(b_s^-(s, \phi, t), t), 12*s*Derivative(\psi(s, \phi, t), \phi)/H(s)**2 + 6*Derivative(\psi(s, \phi, t), \phi, s) - 12*Derivative(\psi(s, \phi, t), \phi)/s)

Eq(Derivative(b_\phi^-(s, \phi, t), t), -6*s**2*Derivative(\psi(s, \phi, t), s)/H(s)**2 - 6*s*Derivative(\psi(s, \phi, t), (s, 2)) + 6*Derivative(\psi(s, \phi, t), s))

Eq(Derivative(b_z^-(s, \phi, t), t), 6*s**2*Derivative(\psi(s, \phi, t), \phi)/H(s)**3 + 6*s*Derivative(\psi(s, \phi, t), \phi, s)/H(s) - 18*Derivative(\psi(s, \phi, t), \phi)/H(s) + 10*Derivative(\psi(s, \phi, t), \phi)/H(s)**3)

In [28]:
ode_s_fd_from_cg = eigen.to_fd_ode_pg(eqs_pg_from_cg, core.pgvar_ptb)

Some further manipulations before output

In [29]:
from pg_utils.sympy_supp import simplify

def common_denom_simplify(expr):
    expr = expr.together()
    num, denom = sp.fraction(expr)
    num = num.subs({core.H: core.H_s}).simplify().factor().subs({core.H_s: core.H, core.H_s**2: core.H**2})
    return num/denom

# eq_grouped = simplify.recursive_collect_tree(eq_rad_std.lhs, (expansion.pgvar_s.Psi, expansion.omega))
# simplify.process_leaf_node(eq_grouped, common_denom_simplify)
# eq_grouped = sp.Eq(simplify.recursive_eval_tree(eq_grouped), eq_rad_std.rhs)

# factor = 1/sp.I/expansion.omega
factor = 1
def factoring(fname, eq):
    if fname != "Psi":
        expr_grouped = (eq.rhs*factor).expand().collect([expansion.pgvar_s.Psi,], evaluate=False)
        for term in expr_grouped:
            expr_grouped[term] = common_denom_simplify(expr_grouped[term])
        return sp.Eq(
            (eq.lhs*factor).expand(), 
            # (eq.rhs*factor).expand().collect(expansion.pgvar_s.Psi)
            factor*sp.Add(*[term*coeff for term, coeff in expr_grouped.items()])
        )
    else:
        expr_grouped = (eq.rhs*factor).expand().collect([var for var in expansion.pgvar_s if var is not None], evaluate=False)
        for term in expr_grouped:
            expr_grouped[term] = common_denom_simplify(expr_grouped[term])
        return sp.Eq((eq.lhs*factor).expand(), sp.Add(*[term*coeff for term, coeff in expr_grouped.items()]))

ode_s_fd_from_cg.apply(factoring, inplace=True, metadata=True)
# display_eq_collection(ode_s_fd)

In [30]:
placeholders_map = base.map_collection(expansion.pgvar_s, base_utils.extract_symbols(core.pgvar_ptb))
placeholder_h = {core.H: sp.Symbol("H")}
for fname in ode_s_fd._field_names:
    if fname == "Br_b":
        continue
    o_str = sp.latex(ode_s_fd_from_cg[fname].subs(placeholder_h))
    for var_old, var_new in placeholders_map.items():
        o_str = o_str.replace(sp.latex(var_old), sp.latex(var_new))
    # print(o_str)
    display(Math(o_str))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

Is the equation equivalent to the one by PG?

In [38]:
for fname in ode_s_fd._field_names:
    if ode_s_fd_from_cg[fname].rhs.subs({core.H: core.H_s}).equals(ode_s_fd[fname].rhs.subs({core.H: core.H_s})) \
        and ode_s_fd_from_cg[fname].lhs.equals(ode_s_fd[fname].lhs):
        display(Markdown("${}$: Equivalent".format(core.pgvar_ptb[fname])))
    else:
        display(Markdown("${}$: NOT equivalent!, difference = ${}$".format(
            core.pgvar_ptb[fname], 
            sp.latex(ode_s_fd_from_cg[fname].rhs.subs({core.H: core.H_s}) - ode_s_fd[fname].rhs.subs({core.H: core.H_s}))
        )))

$\psi(s, \phi, t)$: Equivalent

$\overline{m_{ss}}(s, \phi, t)$: Equivalent

$\overline{m_{\phi\phi}}(s, \phi, t)$: Equivalent

$\overline{m_{s\phi}}(s, \phi, t)$: Equivalent

$\widetilde{m_{sz}}(s, \phi, t)$: Equivalent

...

$b_\phi^+(s, \phi, t)$: Equivalent

$b_z^+(s, \phi, t)$: Equivalent

$b_s^-(s, \phi, t)$: Equivalent

$b_\phi^-(s, \phi, t)$: Equivalent

$b_z^-(s, \phi, t)$: Equivalent

### Reduced dimensional system

In [22]:
str(sp.Rational(1, 2))

'1/2'

In [29]:
eqs_file = "./out/eigen/Toroidal_Quadrupole/eqs_reduced_v0.json"
with open(eqs_file, 'r') as fread:
    eqs_reduced = base.LabeledCollection.deserialize(json.load(fread)["equations"], parser=sp.parse_expr)

display_eq_collection(eqs_reduced)

Eq(s**2*Derivative(\psi(s, \phi, t), s, t)/H(s)**3 + s*Derivative(\psi(s, \phi, t), (s, 2), t)/H(s) + s*Derivative(\psi(s, \phi, t), (\phi, 2), t)/(2*H(s)**3) + Derivative(\psi(s, \phi, t), s, t)/H(s) + Derivative(\psi(s, \phi, t), (\phi, 2), t)/(s*H(s)), F_\mathrm{ext}(s, \phi, t) + 2*s*Derivative(\psi(s, \phi, t), \phi)/(\mathrm{Le}*H(s)**3))

Eq(Derivative(F_\mathrm{ext}(s, \phi, t), t), \gamma**2*s**6*Derivative(\psi(s, \phi, t), (\phi, 2), s)/H(s)**3 + 8*\gamma**2*s**5*Derivative(\psi(s, \phi, t), (\phi, 2), (s, 2))/(15*H(s)) + 2*\gamma**2*s**5*Derivative(\psi(s, \phi, t), (\phi, 2))/H(s)**3 + \gamma**2*s**5*Derivative(\psi(s, \phi, t), (\phi, 4))/(6*H(s)**3) + 8*\gamma**2*s**4*Derivative(\psi(s, \phi, t), (\phi, 2), s)/(3*H(s)) - 2*\gamma**2*s**4*Derivative(\psi(s, \phi, t), (\phi, 2), s)/H(s)**3 + 28*\gamma**2*s**3*Derivative(\psi(s, \phi, t), (\phi, 2))/(15*H(s)) + 8*\gamma**2*s**3*Derivative(\psi(s, \phi, t), (\phi, 4))/(15*H(s)) - 16*\gamma**2*s**3*Derivative(\psi(s, \phi, t), (\phi, 2), (s, 2))/(15*H(s)) - 2*\gamma**2*s**3*Derivative(\psi(s, \phi, t), (\phi, 2))/H(s)**3 - \gamma**2*s**3*Derivative(\psi(s, \phi, t), (\phi, 4))/(3*H(s)**3) - 16*\gamma**2*s**2*Derivative(\psi(s, \phi, t), (\phi, 2), s)/(5*H(s)) + \gamma**2*s**2*Derivative(\psi(s, \phi, t), (\phi, 2), s)/H(s)**3 + 2*\gamma**2*s*Derivative(\psi(s, \phi, 

In [26]:
eqs_file = "./out/eigen/Toroidal_Quadrupole/eqs_reduced.json"
with open(eqs_file, 'r') as fread:
    eqs_reduced = base.LabeledCollection.deserialize(json.load(fread)["equations"], parser=sp.parse_expr)

display_eq_collection(eqs_reduced)

Eq(s**2*Derivative(\psi(s, \phi, t), s, t)/H(s)**3 + s*Derivative(\psi(s, \phi, t), (s, 2), t)/H(s) + s*Derivative(\psi(s, \phi, t), (\phi, 2), t)/(2*H(s)**3) + Derivative(\psi(s, \phi, t), s, t)/H(s) + Derivative(\psi(s, \phi, t), (\phi, 2), t)/(s*H(s)), F_\mathrm{ext}(s, \phi, t) + 2*s*Derivative(\psi(s, \phi, t), \phi)/(\mathrm{Le}*H(s)**3))

Eq(Derivative(F_\mathrm{ext}(s, \phi, t), t), 8*\gamma**2*s**5*Derivative(\psi(s, \phi, t), (\phi, 2), (s, 2))/(15*H(s)) + \gamma**2*s**5*Derivative(\psi(s, \phi, t), (\phi, 4))/(6*H(s)**3) + 5*\gamma**2*s**4*Derivative(\psi(s, \phi, t), (\phi, 2), s)/(3*H(s)) - 32*\gamma**2*s**3*Derivative(\psi(s, \phi, t), (\phi, 2))/(15*H(s)) + 8*\gamma**2*s**3*Derivative(\psi(s, \phi, t), (\phi, 4))/(15*H(s)) - 16*\gamma**2*s**3*Derivative(\psi(s, \phi, t), (\phi, 2), (s, 2))/(15*H(s)) - \gamma**2*s**3*Derivative(\psi(s, \phi, t), (\phi, 4))/(3*H(s)**3) - 11*\gamma**2*s**2*Derivative(\psi(s, \phi, t), (\phi, 2), s)/(5*H(s)) - 2*\gamma**2*s*H(s)*Derivative(\psi(s, \phi, t), (\phi, 2)) + 32*\gamma**2*s*Derivative(\psi(s, \phi, t), (\phi, 2))/(15*H(s)) - 16*\gamma**2*s*Derivative(\psi(s, \phi, t), (\phi, 4))/(15*H(s)) + 8*\gamma**2*s*Derivative(\psi(s, \phi, t), (\phi, 2), (s, 2))/(15*H(s)) + \gamma**2*s*Derivative(\psi(s, \phi, t), (\phi, 4))/(6*H(s)**3) + 8*\gamma**2*Derivative(\psi(s, \phi, t), (\p

In [30]:
eqs_2ord = sp.Eq(sp.diff(eqs_reduced.Psi.lhs, core.t), sp.diff(eqs_reduced.Psi.rhs, core.t).subs({eqs_reduced.F_ext.lhs: eqs_reduced.F_ext.rhs}))
eq_rad = eigen.to_fd_ode_psi(eqs_2ord)
eq_rad = sp.Eq(eq_rad.lhs - eq_rad.rhs, sp.S.Zero)

In [31]:
def common_denom_simplify(expr):
    expr = expr.together()
    num, denom = sp.fraction(expr)
    num = num.subs({core.H: core.H_s}).simplify().factor().subs({core.H_s: core.H, core.H_s**2: core.H**2}).collect(expansion.m)
    return num/denom.factor()

eq_grouped = simplify.recursive_collect_tree(eq_rad.lhs, (expansion.pgvar_s.Psi, expansion.omega, expansion.m))
simplify.process_leaf_node(eq_grouped, common_denom_simplify)
eq_grouped = sp.Eq(simplify.recursive_eval_tree(eq_grouped), eq_rad.rhs)
eq_grouped

Eq((8*\gamma**2*m**2*s*(s - 1)**2*(s + 1)**2/(15*H(s)) - \omega**2*s/H(s))*Derivative(\Psi^{m}(s), (s, 2)) + (-\gamma**2*m**2*(s - 1)**2*(s + 1)**2*(25*s**2 - 8)/(15*H(s)**3) - \omega**2/H(s)**3)*Derivative(\Psi^{m}(s), s) + (\gamma**2*m**4*(s - 1)**2*(s + 1)**2*(11*s**2 - 16)/(30*s*H(s)**3) + 2*\gamma**2*m**2*s*(s - 1)**2*(s + 1)**2/(15*H(s)**3) + \omega**2*m**2*(H(s)**2 + 1)/(2*s*H(s)**3) + 2*\omega*m*s/(\mathrm{Le}*H(s)**3))*\Psi^{m}(s), 0)

In [27]:
eqs_2ord = sp.Eq(sp.diff(eqs_reduced.Psi.lhs, core.t), sp.diff(eqs_reduced.Psi.rhs, core.t).subs({eqs_reduced.F_ext.lhs: eqs_reduced.F_ext.rhs}))
eq_rad = eigen.to_fd_ode_psi(eqs_2ord)
eq_rad = sp.Eq(eq_rad.lhs - eq_rad.rhs, sp.S.Zero)

In [28]:
def common_denom_simplify(expr):
    expr = expr.together()
    num, denom = sp.fraction(expr)
    num = num.subs({core.H: core.H_s}).simplify().factor().subs({core.H_s: core.H, core.H_s**2: core.H**2}).collect(expansion.m)
    return num/denom.factor()

eq_grouped = simplify.recursive_collect_tree(eq_rad.lhs, (expansion.pgvar_s.Psi, expansion.omega, expansion.m))
simplify.process_leaf_node(eq_grouped, common_denom_simplify)
eq_grouped = sp.Eq(simplify.recursive_eval_tree(eq_grouped), eq_rad.rhs)
eq_grouped

Eq((8*\gamma**2*m**2*s*(s - 1)**2*(s + 1)**2/(15*H(s)) - \omega**2*s/H(s))*Derivative(\Psi^{m}(s), (s, 2)) + (\gamma**2*m**2*(s - 1)*(s + 1)*(25*s**2 - 8)/(15*H(s)) - \omega**2/H(s)**3)*Derivative(\Psi^{m}(s), s) + (\gamma**2*m**4*(s - 1)**2*(s + 1)**2*(11*s**2 - 16)/(30*s*H(s)**3) - 2*\gamma**2*m**2*s*(s - 1)*(s + 1)/(15*H(s)) + \omega**2*m**2*(H(s)**2 + 1)/(2*s*H(s)**3) + 2*\omega*m*s/(\mathrm{Le}*H(s)**3))*\Psi^{m}(s), 0)

In [21]:
eqs_2ord = sp.Eq(
    sp.diff(eqs_reduced.Psi.lhs, core.t),
    sp.diff(eqs_reduced.Psi.rhs, core.t).subs({eqs_reduced.F_ext.lhs: eqs_reduced.F_ext.rhs}).doit().expand()
)
eq_rad = eigen.to_fd_ode_psi(eqs_2ord)
eq_rad = sp.Eq(eq_rad.lhs - eq_rad.rhs, sp.S.Zero)

### Second-order form

In [51]:
eqs_file = "./out/eigen/Poloidal_Dipole/eqs_pg.json"
with open(eqs_file, 'r') as fread:
    eqs_pg = base.CollectionPG.deserialize(json.load(fread)["equations"], parser=sp.parse_expr)
eqs_pg_select = eqs_pg.copy()
# for i_eq in range(14, 21):
#     eqs_pg_select[i_eq] = sp.Eq(eqs_pg_select[i_eq].lhs, sp.S.Zero)
eqs_pg_select[16] = sp.Eq(eqs_pg_select[16].lhs, sp.S.Zero)
eqs_pg_select[19] = sp.Eq(eqs_pg_select[19].lhs, sp.S.Zero)

eqs_2ord = eigen.reduce_eqsys_to_psi(eqs_pg_select, verbose=5)
eq_rad = eigen.to_fd_ode_psi(eqs_2ord)
eq_rad = sp.Eq(eq_rad.lhs - eq_rad.rhs, sp.S.Zero)

========== Converting to 2nd order dynamical system... ==========
========== Reducing dimension of dynamical system... ==========
Extracting body forces...
Forming dynamical system of Psi and F...


In [43]:
eqs_pg_select[0]

Eq(s**2*Derivative(\psi(s, \phi, t), s, t)/H(s)**3 + s*Derivative(\psi(s, \phi, t), (s, 2), t)/H(s) + s*Derivative(\psi(s, \phi, t), (\phi, 2), t)/(2*H(s)**3) + Derivative(\psi(s, \phi, t), s, t)/H(s) + Derivative(\psi(s, \phi, t), (\phi, 2), t)/(s*H(s)), -12*s**4*b_{\phi, z}^e(s, \phi, t)/H(s)**2 - 3*s**4*Derivative(b_s^+(s, \phi, t), \phi)/H(s)**3 + 3*s**4*Derivative(b_s^-(s, \phi, t), \phi)/H(s)**3 - 3*s**3*Derivative(b_z^+(s, \phi, t), \phi)/H(s)**2 - 3*s**3*Derivative(b_z^-(s, \phi, t), \phi)/H(s)**2 + 24*s**3*Derivative(b_{z}^e(s, \phi, t), \phi)/H(s)**2 - 3*s**2*Derivative(b_s^+(s, \phi, t), \phi)/H(s) + 3*s**2*Derivative(b_s^-(s, \phi, t), \phi)/H(s) + 10*s**2*b_{\phi, z}^e(s, \phi, t)/H(s)**2 + 2*s**2*Derivative(b_s^+(s, \phi, t), \phi)/H(s)**3 - 2*s**2*Derivative(b_s^-(s, \phi, t), \phi)/H(s)**3 - 3*s*Derivative(b_z^+(s, \phi, t), \phi) - 3*s*Derivative(b_z^-(s, \phi, t), \phi) - s*Derivative(\overline{m_{s\phi}}(s, \phi, t), (s, 2))/(2*H(s)) + 2*s*Derivative(b_\phi^+(s, \phi

Further simplifications: standard ODE form

In [44]:
eq_rad

Eq(\omega**2*m**2*s*\Psi^{m}(s)/(2*H(s)**3) + \omega**2*m**2*\Psi^{m}(s)/(s*H(s)) - \omega**2*s**2*Derivative(\Psi^{m}(s), s)/H(s)**3 - \omega**2*s*Derivative(\Psi^{m}(s), (s, 2))/H(s) - \omega**2*Derivative(\Psi^{m}(s), s)/H(s) - 36*m**2*s**6*Derivative(\Psi^{m}(s), s)/H(s)**5 - 36*m**2*s**5*Derivative(\Psi^{m}(s), (s, 2))/H(s)**3 + 486*m**2*s**5*\Psi^{m}(s)/H(s)**5 - 45*m**2*s**4*Derivative(\Psi^{m}(s), s)/H(s)**3 + 57*m**2*s**4*Derivative(\Psi^{m}(s), s)/H(s)**5 - 39*m**2*s**3*Derivative(\Psi^{m}(s), (s, 2))/H(s) + 720*m**2*s**3*\Psi^{m}(s)/H(s)**3 + 57*m**2*s**3*Derivative(\Psi^{m}(s), (s, 2))/H(s)**3 - 807*m**2*s**3*\Psi^{m}(s)/H(s)**5 + 3*m**2*s**2*Derivative(\Psi^{m}(s), s)/H(s) + 45*m**2*s**2*Derivative(\Psi^{m}(s), s)/H(s)**3 - 21*m**2*s**2*Derivative(\Psi^{m}(s), s)/H(s)**5 + 324*m**2*s*\Psi^{m}(s)/H(s) + 33*m**2*s*Derivative(\Psi^{m}(s), (s, 2))/H(s) - 450*m**2*s*\Psi^{m}(s)/H(s)**3 - 21*m**2*s*Derivative(\Psi^{m}(s), (s, 2))/H(s)**3 + 402*m**2*s*\Psi^{m}(s)/H(s)**5 + 15*m**

In [52]:
ord_max = 4
leading_cf = eq_rad.lhs.coeff(sp.diff(expansion.pgvar_s.Psi, (core.s, ord_max))).together().subs({core.H_s: core.H, core.H_s**2: core.H**2})
eq_rad_std = sp.Eq((eq_rad.lhs/leading_cf).expand(), eq_rad.rhs/leading_cf)
sp.Eq(eq_rad_std.lhs.collect(expansion.pgvar_s.Psi), eq_rad_std.rhs)

Eq((-9*s/H(s)**2 + 6/(s*H(s)**2))*Derivative(\Psi^{m}(s), (s, 3)) + (-s**2/H(s)**2 + H(s)**(-2))*Derivative(\Psi^{m}(s), (s, 4)) + (\omega**2/(12*s**2*H(s)**2) + 3*m**2*s**2/H(s)**4 + 13*m**2/(4*H(s)**2) - 19*m**2/(4*H(s)**4) - 11*m**2/(4*s**2*H(s)**2) + 7*m**2/(4*s**2*H(s)**4) - 6*s**2/H(s)**4 - 6/H(s)**2 + 5/H(s)**4 + 3/(s**2*H(s)**2))*Derivative(\Psi^{m}(s), (s, 2)) + (-\omega**2*m**2/(24*s**2*H(s)**4) - \omega**2*m**2/(12*s**4*H(s)**2) - 81*m**2*s**2/(2*H(s)**6) - 60*m**2/H(s)**4 + 269*m**2/(4*H(s)**6) - 27*m**2/(s**2*H(s)**2) + 75*m**2/(2*s**2*H(s)**4) - 67*m**2/(2*s**2*H(s)**6) + 8*m**2/(s**4*H(s)**2) - \omega*m/(6*\mathrm{Le}*s**2*H(s)**4))*\Psi^{m}(s) + (\omega**2/(12*s*H(s)**4) + \omega**2/(12*s**3*H(s)**2) + 3*m**2*s**3/H(s)**6 + 15*m**2*s/(4*H(s)**4) - 19*m**2*s/(4*H(s)**6) - m**2/(4*s*H(s)**2) - 15*m**2/(4*s*H(s)**4) + 7*m**2/(4*s*H(s)**6) - 5*m**2/(4*s**3*H(s)**2) - 7*m**2/(4*s**3*H(s)**4) - 6*s**3/H(s)**6 + 6*s/H(s)**4 + 5*s/H(s)**6 + 30/(s*H(s)**2) - 5/(s*H(s)**4) - 3/(s

Group the coefficients

In [54]:
from pg_utils.sympy_supp import simplify

def common_denom_simplify(expr):
    expr = expr.together()
    num, denom = sp.fraction(expr)
    num = num.subs({core.H: core.H_s}).simplify().factor().subs({core.H_s: core.H, core.H_s**2: core.H**2})
    return num/denom

eq_grouped = simplify.recursive_collect_tree(eq_rad_std.lhs, (expansion.pgvar_s.Psi, expansion.omega))
simplify.process_leaf_node(eq_grouped, common_denom_simplify)
eq_grouped = sp.Eq(simplify.recursive_eval_tree(eq_grouped), eq_rad_std.rhs)
eq_grouped

Eq(-(s - 1)*(s + 1)*Derivative(\Psi^{m}(s), (s, 4))/H(s)**2 + (\omega**2/(12*s**3*H(s)**4) - (m**2*s**6 - 2*m**2*s**4 - 2*m**2*s**2 + 3*m**2 - 18*s**6 + 47*s**4 - 31*s**2 + 3)/(s**3*H(s)**6))*Derivative(\Psi^{m}(s), s) + (\omega**2/(12*s**2*H(s)**2) + (-m**2*s**4 + 5*m**2*s**2 - 4*m**2 - 16*s**2 + 12)/(4*s**2*H(s)**4))*Derivative(\Psi^{m}(s), (s, 2)) + (\omega**2*m**2*(-H(s)**2 - 1)/(24*s**4*H(s)**4) - m**2*(30*s**6 - 127*s**4 + 156*s**2 - 32)/(4*s**4*H(s)**6) - \omega*m/(6*\mathrm{Le}*s**2*H(s)**4))*\Psi^{m}(s) - 3*(3*s**2 - 2)*Derivative(\Psi^{m}(s), (s, 3))/(s*H(s)**2), 0)

Output

In [23]:
sp.print_latex(eq_grouped)

\left(\frac{m^{2} \left(- H^{2}{\left(s \right)} - 1\right)}{2 s^{2} H^{2}{\left(s \right)}} - \frac{2 m}{\omega H^{2}{\left(s \right)}}\right) \Psi^{m}{\left(s \right)} + \frac{d^{2}}{d s^{2}} \Psi^{m}{\left(s \right)} + \frac{\frac{d}{d s} \Psi^{m}{\left(s \right)}}{s H^{2}{\left(s \right)}} = 0


## Matrix elements

In [71]:
from pg_utils.numerics import matrices as nmatrix

matrix_file = "./out/eigen/Malkus/Transformed/matrix_expr.json"
with open(matrix_file, 'r') as fread:
    matrix_obj = json.load(fread)

M_expr = expansion.SystemMatrix.deserialize(matrix_obj["M"])
K_expr = expansion.SystemMatrix.deserialize(matrix_obj["K"])
par_list = [sp.parse_expr(par) for par in matrix_obj["params"]]

In [83]:
iprod = nmatrix.InnerQuad_GaussJacobi(M_expr["B_em", "B_em"], automatic=True)

In [85]:
iprod.deduce_params_outer(51, 51)

(0, Abs(m - 1) - 1/2, 52, 0, Abs(m - 1)/2)

In [88]:
M_expr["B_em", "B_em"]._opd_A

(\xi/2 + 1/2)**(Abs(m - 1)/2)*jacobi(\ell', 0, Abs(m - 1) - 1/2, \xi)

In [87]:
nmatrix.InnerQuad_GaussJacobi.get_powers(expansion.xi, M_expr["B_em", "B_em"]._opd_A.doit())

[0, Abs(m - 1)/2, \ell']

In [74]:
opd_A_terms = nmatrix.InnerQuad_GaussJacobi.get_powers(expansion.xi, K_expr["F_ext", "Psi"]._opd_A, return_expr=True)
if not isinstance(opd_A_terms, list):
    opd_A_terms = [opd_A_terms]

for term in opd_A_terms:
    display(term[1])
    print(term[0])

sqrt(2)*sqrt(p_2)*p_2**(Abs(m)/2)*jacobi(\ell', 0, Abs(m) + 1/2, \xi)/(2*2**(Abs(m)/2))

[0, Abs(m)/2 + 1/2, \ell']


In [75]:
opd_B_terms = nmatrix.InnerQuad_GaussJacobi.get_powers(expansion.xi, K_expr["F_ext", "Psi"]._opd_B.doit(), return_expr=True)
if not isinstance(opd_B_terms, list):
    opd_B_terms = [opd_B_terms]

In [76]:
for term in opd_B_terms:
    if term[0][0] > -1:
        continue
    display(term[1])
    print(term[0])

-7*\gamma**2*m**2*p_2**2*p_2**(Abs(m)/2)*jacobi(\ell, 3/2, Abs(m), \xi)/(20*2**(Abs(m)/2)*p_1)

[-1, Abs(m)/2 + 2, \ell]


7*\gamma**2*m**2*p_2*p_2**(Abs(m)/2)*jacobi(\ell, 3/2, Abs(m), \xi)/(20*2**(Abs(m)/2)*p_1)

[-1, Abs(m)/2 + 1, \ell]


7*\gamma**2*m**2*p_2**3*p_2**(Abs(m)/2)*jacobi(\ell, 3/2, Abs(m), \xi)/(80*2**(Abs(m)/2)*p_1)

[-1, Abs(m)/2 + 3, \ell]


In [77]:
term_singular = sp.S.Zero
for term in opd_B_terms:
    if term[0][0] <= -1:
        term_singular += term[1]

In [78]:
term_singular

7*\gamma**2*m**2*p_2**3*p_2**(Abs(m)/2)*jacobi(\ell, 3/2, Abs(m), \xi)/(80*2**(Abs(m)/2)*p_1) - 7*\gamma**2*m**2*p_2**2*p_2**(Abs(m)/2)*jacobi(\ell, 3/2, Abs(m), \xi)/(20*2**(Abs(m)/2)*p_1) + 7*\gamma**2*m**2*p_2*p_2**(Abs(m)/2)*jacobi(\ell, 3/2, Abs(m), \xi)/(20*2**(Abs(m)/2)*p_1)

In [81]:
term_singular.subs({sp.Symbol(r"p_2", positive=True): 2 - sp.Symbol(r"p_1", positive=True)}).simplify().subs({2 - sp.Symbol(r"p_1", positive=True): sp.Symbol(r"p_2", positive=True)})

7*\gamma**2*m**2*p_1*p_2**(Abs(m)/2 + 1)*jacobi(\ell, 3/2, Abs(m), \xi)/(80*2**(Abs(m)/2))

In [83]:
[term[0] for term in opd_B_terms]

[[0, Abs(m)/2, \ell],
 [0, Abs(m)/2, \ell],
 [0, Abs(m)/2 + 1, \ell],
 [0, Abs(m)/2 + 2, \ell - 1],
 [1, Abs(m)/2, \ell - 1],
 [1, Abs(m)/2, \ell - 1],
 [0, Abs(m)/2 + 1, \ell],
 [1, Abs(m)/2, \ell],
 [0, Abs(m)/2 + 2, \ell],
 [0, Abs(m)/2 + 1, \ell - 1],
 [0, Abs(m)/2, \ell],
 [0, Abs(m)/2 + 3, \ell - 1],
 [0, Abs(m)/2 + 2, \ell],
 [1, Abs(m)/2, \ell - 1],
 [0, Abs(m)/2 + 2, \ell - 1],
 [0, Abs(m)/2 + 1, \ell],
 [0, Abs(m)/2 + 2, \ell - 1],
 [1, Abs(m)/2 + 1, \ell - 2],
 [-1, Abs(m)/2 + 2, \ell],
 [1, Abs(m)/2 + 3, \ell - 2],
 [1, Abs(m)/2 + 1, \ell],
 [1, Abs(m)/2, \ell - 1],
 [1, Abs(m)/2 + 1, \ell - 2],
 [1, Abs(m)/2 + 2, \ell - 1],
 [1, Abs(m)/2 + 2, \ell - 1],
 [1, Abs(m)/2 + 3, \ell - 2],
 [1, Abs(m)/2 + 2, \ell - 2],
 [1, Abs(m)/2 + 1, \ell - 1],
 [1, Abs(m)/2 + 1, \ell - 1],
 [1, Abs(m)/2, \ell],
 [1, Abs(m)/2 + 2, \ell - 2],
 [-1, Abs(m)/2 + 1, \ell],
 [-1, Abs(m)/2 + 3, \ell],
 [0, Abs(m)/2 + 1, \ell - 1],
 [0, Abs(m)/2 + 1, \ell - 1],
 [0, Abs(m)/2 + 3, \ell - 1],
 [0, Abs(

In [57]:
with open("./out/symbolic/eqs_pg_lin.json", 'r') as fread:
    eqs = base.LabeledCollection.load_json(fread, parser=sp.parse_expr)

In [58]:
eqs.Psi

Eq(s*Derivative(\psi(s, \phi, t), (s, 2), t)/H(s) - s*Derivative(H(s), s)*Derivative(\psi(s, \phi, t), s, t)/H(s)**2 + Derivative(\psi(s, \phi, t), s, t)/H(s) - Derivative(H(s), s)*Derivative(\psi(s, \phi, t), (\phi, 2), t)/(2*H(s)**2) + Derivative(\psi(s, \phi, t), (\phi, 2), t)/(s*H(s)), s*f_{\phi}^e(s, \phi, t)*Derivative(H(s), s)/H(s) - s*Derivative(\overline{f_\phi}(s, \phi, t), s)/(2*H(s)) - \overline{f_\phi}(s, \phi, t)/(2*H(s)) + Derivative(H(s), s)*Derivative(\widetilde{f_z}(s, \phi, t), \phi)/(2*H(s)) + Derivative(\overline{f_s}(s, \phi, t), \phi)/(2*H(s)) - 2*Derivative(H(s), s)*Derivative(\psi(s, \phi, t), \phi)/H(s)**2)

In [29]:
eqs.dBp_dz_e

Eq(Derivative(b_{\phi, z}^e(s, \phi, t), t), -B_{s, z}^{0e}(s, \phi)*Derivative(\psi(s, \phi, t), (s, 2))/H(s) + B_{s, z}^{0e}(s, \phi)*Derivative(H(s), s)*Derivative(\psi(s, \phi, t), s)/H(s)**2 - U_s^0(s, \phi, z)*Derivative(b_{\phi, z}^e(s, \phi, t), s) - b_{\phi, z}^e(s, \phi, t)*Derivative(U_z^0(s, \phi, z), z) + b_{s, z}^e(s, \phi, t)*Derivative(U_\phi^0(s, \phi, z), s) - B_{\phi, z}^{0e}(s, \phi)*Derivative(\psi(s, \phi, t), \phi, s)/(s*H(s)) - B_{\phi, z}^{0e}(s, \phi)*Derivative(H(s), s)*Derivative(\psi(s, \phi, t), \phi)/(s*H(s)**2) + B_{s, z}^{0e}(s, \phi)*Derivative(\psi(s, \phi, t), s)/(s*H(s)) - U_\phi^0(s, \phi, z)*b_{s, z}^e(s, \phi, t)/s - U_\phi^0(s, \phi, z)*Derivative(b_{\phi, z}^e(s, \phi, t), \phi)/s + U_s^0(s, \phi, z)*b_{\phi, z}^e(s, \phi, t)/s + b_{\phi, z}^e(s, \phi, t)*Derivative(U_\phi^0(s, \phi, z), \phi)/s + Derivative(B_{\phi, z}^{0e}(s, \phi), \phi)*Derivative(\psi(s, \phi, t), s)/(s*H(s)) - Derivative(B_{\phi, z}^{0e}(s, \phi), s)*Derivative(\psi(s, \p

In [48]:
(sp.diff(core.s*sp.diff(core.s**expansion.m*core.H_s**3, core.s), core.s)/expansion.H_s).expand()

-m**2*s*s**m + m**2*s**m/s - 6*m*s*s**m + 3*s**3*s**m/(1 - s**2) - 6*s*s**m

In [46]:
sp.diff(core.s**expansion.m, core.s).subs({expansion.m: 0})

0

In [50]:
(sp.diff(core.s**expansion.m*core.H_s**3, core.s)/expansion.H_s).expand()

-m*s*s**m + m*s**m/s - 3*s*s**m